In [1]:
# Modules
import numpy as np
import pandas as pd
import ast


In [2]:
def convertStringList_to_List(text):
    L = []
    # ast.literal_eval  ->  to convert string list to list
    for i in ast.literal_eval(text): 
        L.append(i['name']) 
    return L 

# for fetching top three actors of movies from list
def fetchLearActorNames(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

# for fetching direcor of movies from list 
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [3]:
# importing all files from directories

movies=pd.read_csv('./movies.csv.csv',low_memory=False)
credits=pd.read_csv('./credits.csv')
ratings=pd.read_csv('./ratings.csv')
links=pd.read_csv('./links.csv')
keywords=pd.read_csv('./keywords.csv')
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
#rename tmbdId to id for merge it with moveis and also convert float value to string
links.rename(columns = {'tmdbId':'id'}, inplace = True)
links.id=links.id.astype(str)
links.id=links.id.apply(lambda x:x[:len(x)-2])
links

,movieId,imdbId,id
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862
...,...,...,...
45838,176269,6209470,439050
45839,176271,2028550,111109
45840,176273,303758,67758
45841,176275,8536,227506


In [6]:
#filter out necessary columns
movies=movies[['genres','id','imdb_id','overview','tagline','title','vote_average','vote_count']]
movies

,genres,id,imdb_id,overview,tagline,title,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",NaN,Toy Story,7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...
45461,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,tt6209470,Rising and falling between a man and woman.,Rising and falling between a man and woman,Subdue,4.0,1.0
45462,"[{'id': 18, 'name': 'Drama'}]",111109,tt2028550,An artist struggles to finish his work while a...,NaN,Century of Birthing,9.0,3.0
45463,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,tt0303758,"When one of her hits goes wrong, a professiona...",A deadly game of wits.,Betrayal,3.8,6.0
45464,[],227506,tt0008536,"In a small town live two brothers, one a minis...",NaN,Satan Triumphant,0.0,0.0


In [8]:
# now filter out those users which are rated more than 200 movies 
ratings_small=ratings['userId'].value_counts()>500
ratings_small=ratings_small[ratings_small]
temp=[]
for idx in pd.DataFrame(ratings_small).index:
    temp.append(idx)
temp=pd.DataFrame(set(temp),columns=['userId'])
ratings_small=temp
ratings_small=ratings_small.merge(pd.DataFrame(ratings),on='userId',how='inner')

9452

In [9]:
# convert cast into list of top 3 actors and director  and then drop crew column
credits['cast']=credits['cast'].apply(fetchLearActorNames)
credits['crew']=credits['crew'].apply(fetch_director)
credits['cast']=credits['crew']+credits['cast']
credits.drop(credits.columns[1], axis=1, inplace=True)
credits


,cast,id
0,"[John Lasseter, Tom Hanks, Tim Allen, Don Rick...",862
1,"[Joe Johnston, Robin Williams, Jonathan Hyde, ...",8844
2,"[Howard Deutch, Walter Matthau, Jack Lemmon, A...",15602
3,"[Forest Whitaker, Whitney Houston, Angela Bass...",31357
4,"[Charles Shyer, Steve Martin, Diane Keaton, Ma...",11862
...,...,...
45471,"[Hamid Nematollah, Leila Hatami, Kourosh Taham...",439050
45472,"[Lav Diaz, Angel Aquino, Perry Dizon, Hazel Or...",111109
45473,"[Mark L. Lester, Erika Eleniak, Adam Baldwin, ...",67758
45474,"[Yakov Protazanov, Iwan Mosschuchin, Nathalie ...",227506


In [10]:
# convert keywords from list of objects to list of keys
keywords['keywords']=keywords['keywords'].apply(convertStringList_to_List)
keywords

,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."
...,...,...
46414,439050,[tragic love]
46415,111109,"[artist, play, pinoy]"
46416,67758,[]
46417,227506,[]


In [10]:
# convert genres to list of genre 
movies['genres']=movies['genres'].apply(convertStringList_to_List)
movies

C:\Users\acer\AppData\Local\Temp\ipykernel_20840\2890825111.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres']=movies['genres'].apply(convertStringList_to_List)


,genres,id,imdb_id,overview,tagline,title,vote_average,vote_count
0,"[Animation, Comedy, Family]",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",NaN,Toy Story,7.7,5415.0
1,"[Adventure, Fantasy, Family]",8844,tt0113497,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"[Romance, Comedy]",15602,tt0113228,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,"[Comedy, Drama, Romance]",31357,tt0114885,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,[Comedy],11862,tt0113041,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...
45461,"[Drama, Family]",439050,tt6209470,Rising and falling between a man and woman.,Rising and falling between a man and woman,Subdue,4.0,1.0
45462,[Drama],111109,tt2028550,An artist struggles to finish his work while a...,NaN,Century of Birthing,9.0,3.0
45463,"[Action, Drama, Thriller]",67758,tt0303758,"When one of her hits goes wrong, a professiona...",A deadly game of wits.,Betrayal,3.8,6.0
45464,[],227506,tt0008536,"In a small town live two brothers, one a minis...",NaN,Satan Triumphant,0.0,0.0


In [12]:
# merging links with movies
movies=movies.merge(links,on='id')

In [13]:
# Remove nan values from taglines and overview
def remove_nan_values(text):
    text=str(text)
    if text=='nan':
        return ""
    return (text)
movies.tagline=movies.tagline.apply(remove_nan_values)
movies.overview=movies.overview.apply(remove_nan_values)

# remove all row which have any null values 
movies.dropna(inplace=True)
movies.isnull().sum()

genres          0
id              0
imdb_id         0
overview        0
tagline         0
title           0
vote_average    0
vote_count      0
movieId         0
imdbId          0
dtype: int64

In [14]:
# convert id of credits and keywords to string
credits.id=credits.id.astype(str)
keywords.id=keywords.id.astype(str)
movies

,genres,id,imdb_id,overview,tagline,title,vote_average,vote_count,movieId,imdbId
0,"[Animation, Comedy, Family]",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",,Toy Story,7.7,5415.0,1,114709
1,"[Adventure, Fantasy, Family]",8844,tt0113497,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,2,113497
2,"[Romance, Comedy]",15602,tt0113228,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,3,113228
3,"[Comedy, Drama, Romance]",31357,tt0114885,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,4,114885
4,[Comedy],11862,tt0113041,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,5,113041
...,...,...,...,...,...,...,...,...,...,...
45520,"[Drama, Family]",439050,tt6209470,Rising and falling between a man and woman.,Rising and falling between a man and woman,Subdue,4.0,1.0,176269,6209470
45521,[Drama],111109,tt2028550,An artist struggles to finish his work while a...,,Century of Birthing,9.0,3.0,176271,2028550
45522,"[Action, Drama, Thriller]",67758,tt0303758,"When one of her hits goes wrong, a professiona...",A deadly game of wits.,Betrayal,3.8,6.0,176273,303758
45523,[],227506,tt0008536,"In a small town live two brothers, one a minis...",,Satan Triumphant,0.0,0.0,176275,8536


In [15]:
# merge credits and keywords into movies
movies=movies.merge(credits,on='id')
movies=movies.merge(keywords,on='id')

In [16]:
# merge filtered ratings into movies
movies=movies.merge(ratings_small,on='movieId')

# remove all row which have any null values 
movies.dropna(inplace=True)
movies.isnull().sum()

MemoryError: Unable to allocate 240. MiB for an array with shape (2, 15749428) and data type int64

In [24]:
import pickle

# create a file whcih containing data of movies dataframe with name
pickle.dump(movies,open('movies_meta_data.pkl','wb'))